In [1]:
import scala.util.{Failure, Success, Try}

sealed trait Location
case object School extends Location
case object Home extends Location
case object Restaurant extends Location

case class Motorcycle(fuel: Int, location: Location)
case class Person(name: String, location: Location)
case class PositionUpdate(joe: Location, sam: Location)

object TravelFunctions {
  val tripCost = 20

  def drive(motorcycle: Motorcycle, destination: Location): Try[Motorcycle] = {
    motorcycle match {
      case Motorcycle(_, location) if (location == destination) => Success(motorcycle)
      case Motorcycle(fuel, _) if (fuel >= tripCost) => Success(Motorcycle(motorcycle.fuel - tripCost, destination))
      case _ => Failure(new Exception("Ran out of gas!"))
    }
  }

  def drive(person: Person, motorcycle: Motorcycle, destination: Location): Try[(Person, Motorcycle)] = {
    person.location match {
      case `destination` => Success((person, motorcycle)) // Try to ignore the backticks
      case motorcycle.location =>
        TravelFunctions.drive(motorcycle, destination) flatMap { movedCar =>
          Success((person.copy(location = destination), movedCar))
        }
      case invalidLocation => Failure(new Exception("Car and driver aren't in the same place!"))
    }
  }

  def fill(person: Person, motorcycle: Motorcycle): Try[Motorcycle] =
    person.location match {
      case motorcycle.location => Success(motorcycle.copy(fuel = 100))
      case _ => Failure(new Exception("Car and driver aren't in the same place!"))
    }
}

object Scenarios {

  def updateScene(joe: Person, sam: Person, motorcycle: Motorcycle, intentions: PositionUpdate): Try[(Person, Person, Motorcycle)] = {
    for ((newJoe, newCar) <- TravelFunctions.drive(joe, motorcycle, intentions.joe);
         (newSam, finalCar) <- TravelFunctions.drive(sam, newCar, intentions.sam) ) yield {
      (newJoe, newSam, finalCar)
    }
  }

  def processMovements(joe: Person, sam: Person, motorcycle: Motorcycle, intentions: PositionUpdate*): Try[(Person, Person, Motorcycle)] =
    processMovements(joe, sam, motorcycle, intentions.toList)

  def processMovements(joe: Person, sam: Person, motorcycle: Motorcycle, intentions: List[PositionUpdate]): Try[(Person, Person, Motorcycle)] = {
    intentions.foldLeft(Try((joe, sam, motorcycle))) {
      case (Success((curJoe, curSam, curMotorcycle)), curIntentions) => updateScene(curJoe, curSam, curMotorcycle, curIntentions)
      case (Failure(ex), curIntentions) => Failure(ex)
    }
  }
}

import scala.util.{Failure, Success, Try}
defined trait Location
defined object School
defined object Home
defined object Restaurant
defined class Motorcycle
defined class Person
defined class PositionUpdate
defined object TravelFunctions
defined object Scenarios

In [2]:
val SAM = Person("Sam", Home)
val JOE = Person("Joe", Home)
val CAR = Motorcycle(100, Home)


SAM: Person = Person(Sam,Home)
JOE: Person = Person(Joe,Home)
CAR: Motorcycle = Motorcycle(100,Home)

In [ ]:
CAR.copy(fuel=80)

In [ ]:
TravelFunctions.drive(SAM, CAR, School)